# Tasks 

## Task 1: Binary Representatios

Focusing on implemnet the four key bitwise operations, 


1. `rotl(x, n=1)` – Bit rotation to the left.
2. `rotr(x, n=1)` – Bit rotation to the right.
3. `ch(x, y, z)` – Choose function: bits from y or z depending on x.
4. `maj(x, y, z)` – Majority function based on bit voting.

All operations assume 32-bit unsigned integrers 

### Creating the function for the bit rotation to the right

In [8]:
# Function for roating left 
def rotl(x, n=1, bits=32):
    return ((x << n) & (2**bits - 1)) | (x >> (bits - n))

### rotl(x, n)

The `rotl` function performs a circular left shift on a 32-bit integer.
- Bits that go past the left edge wrap around to the right side.
- We apply a bitmask `& (2**32 - 1)` to ensure the result stays within 32 bits.

Example:
- `rotl(0b1001, 1)` → shifts left and wraps `1` around.


### Creating the function for bit rotation to the left

In [9]:
# Function for roating right
def rotr(x, n=1, bits=32):
    return (x >> n) | ((x << (bits - n)) & (2**bits - 1))

### rotr(x, n)

`rotr` rotates the bits to the right — the inverse of `rotl`.
- High bits from the left wrap around to the right.
- Useful in SHA and cryptographic transforms.

Example:
- `rotr(0b1001, 1)` → last bit comes to front.


In [10]:
def ch(x, y, z):
    return (x & y) ^ (~x & z)



### ch(x, y, z)

This function is often used in SHA-2.
- It's a conditional selector at the bit level.
- If a bit in `x` is 1 → result takes the corresponding bit from `y`.
- If 0 → it takes it from `z`.

It's like `x ? y : z` for each bit.


In [11]:
def maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)


### maj(x, y, z)

This function returns the majority bit for each position:
- If 2 or more of `x`, `y`, `z` have a `1`, the result has a `1` in that bit.
- It's used in SHA256 to mix entropy.

Mathematically: `maj(x, y, z) = (x & y) ^ (x & z) ^ (y & z)`


### Test for part 1 

In [12]:
# Example test values
x = 0b10101010_11110000_00001111_11000011
y = 0b11001100_00110011_11110000_00001111
z = 0b11110000_00001111_10101010_01010101

print(f"rotl(x, 3): {rotl(x, 3):032b}")
print(f"rotr(x, 3): {rotr(x, 3):032b}")
print(f"ch(x, y, z): {ch(x, y, z):032b}")
print(f"maj(x, y, z): {maj(x, y, z):032b}")


rotl(x, 3): 01010111100000000111111000011101
rotr(x, 3): 01110101010111100000000111111000
ch(x, y, z): 11011000001111111010000000010111
maj(x, y, z): 11101000001100111010101001000111


## Task 2

## Task 3

## Task 4

## Task 5

## Task 6

## Task 7